## Device-Associated Infections Hyperparametric Search

### Load data and variable selection

In [45]:
X_train=X_train[[
'imv','iuc','cvc','gender','myocardial_infarct','malignant_cancer','liver_disease','case_micu/sicu'
]]

X_test=X_test[[
'imv','iuc','cvc','gender','myocardial_infarct','malignant_cancer','liver_disease','case_micu/sicu'
]]

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from scipy import interp
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.calibration import CalibrationDisplay
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import export_graphviz
import graphviz
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn import tree
from sklearn.linear_model import SGDClassifier
import xgboost as xgb
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from sklearn.metrics import PrecisionRecallDisplay

### BayesianOptimization for RF LR SVM XGBoost GNB DT 

In [ ]:
### We initially narrowed down the parameters through a manual search

In [ ]:
### BayesianOptimization code for RF LR SVM XGBoost GNB DT
# SVM example  
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import warnings  
warnings.filterwarnings(action='ignore')  


def rf_cv(max_iter):
    val = cross_val_score(
       SVC(probability= True, 
           kernel='linear',
           degree=3,
           max_iter=int(max_iter),
           random_state=123
           ), 
        X_train, y_train, scoring='neg_mean_squared_error', cv=10
    ).mean()
    return val

# 规定各参数搜索范围
rf_bo = BayesianOptimization(rf_cv,
                             {'max_iter': (..., ...)
                              })
rf_bo.maximize()

rf_bo.max

### BayesianOptimization for LSTM

In [ ]:
from bayes_opt import BayesianOptimization  
from sklearn.model_selection import train_test_split  
import warnings  
import matplotlib.pyplot as plt  
import seaborn as sns 
import pandas as pd  
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, r2_score  # 模型评估方法
import keras.layers as layers  
import keras.backend as K  
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  

from tensorflow.keras.utils import plot_model  
 
warnings.filterwarnings(action='ignore')  
 
 

def bayesopt_objective_lstm(units, epochs):
    lstm = Sequential() 
    lstm.add(LSTM(units=32, return_sequences=True, input_shape=(X_train.shape[1], 1))) 
    lstm.add(LSTM(units=int(units))) 
    lstm.add(Dense(10, activation='relu'))  
    lstm.add(Dense(1))  
    lstm.compile(loss='mean_squared_error',
                 optimizer='adam', metrics='mse')  
    lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=int(epochs), batch_size=64)  
    score = lstm.evaluate(X_test, y_test, batch_size=128) 
 
    return score[1] 
 
 

def param_bayes_opt_lstm(init_points, n_iter):
    opt = BayesianOptimization(bayesopt_objective_lstm
                               , param_grid_simple
                               , random_state=7) 
 
  
    opt.maximize(init_points=init_points  
                 , n_iter=n_iter  
                 )
 
    
    params_best = opt.max['params'] 
    score_best = opt.max['target']  
 
    return params_best, score_best 
 
 

def bayes_opt_validation_lstm(params_best):
    lstm = Sequential() 
    lstm.add(LSTM(units=32, return_sequences=True, input_shape=(X_train.shape[1], 1))) 
    lstm.add(LSTM(units=int(params_best['units'])))
    lstm.add(Dense(10, activation='relu')) 
    lstm.add(Dense(1)) 
    lstm.compile(loss='mean_squared_error',
                 optimizer='adam',
                 metrics=['mse']) 
    lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=int(params_best['epochs']), batch_size=64) 
 
    score = lstm.evaluate(X_test, y_test, batch_size=128) 
 
    return score[1] 
 
 

if __name__ == '__main__':
    
 
 
 
    X_train = layers.Lambda(lambda X_trainf: K.expand_dims(X_trainf, axis=-1))(X_trainf) 
 
    print('***********************查看训练集的形状**************************')
    print(X_train.shape)  
 
    X_test = layers.Lambda(lambda X_testf: K.expand_dims(X_testf, axis=-1))(X_testf)  
    print('***********************查看测试集的形状**************************')
    print(X_test.shape)  
 
    param_grid_simple = {'units': (..., ...)  
        , 'epochs': (..., ...)  
                         }
 
    params_best, score_best = param_bayes_opt_lstm(10, 10)  
 
    print('最优参数组合:  ', 'epochs的参数值为：', int(params_best['epochs']), '  units的参数值为：',
          int(params_best['units']))
    print('最优分数：  ', abs(score_best))  
    validation_score = bayes_opt_validation_lstm(params_best)  
    print('验证集MSE：  ', abs(validation_score)) 
 
  
    lstm = Sequential()  
    lstm.add(LSTM(units=32, return_sequences=True, input_shape=(X_train.shape[1], 1)))  
    lstm.add(LSTM(int(params_best['units']))) 
    lstm.add(Dense(10, activation='relu')) 
    lstm.add(Dense(1)) 
    lstm.compile(loss='mean_squared_error',
                 optimizer='adam',
                 metrics=['mse']) 
    history = lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=int(params_best['epochs']),
                       batch_size=64) 
    print('*************************输出模型摘要信息*******************************')
    print(lstm.summary())  
 
    plot_model(lstm, to_file='model.png', show_shapes=True)

In [ ]:
print('最优参数组合:  ', 'epochs的参数值为：', int(params_best['epochs']), '  units的参数值为：',
          int(params_best['units'])) 
print('最优分数：  ', abs(score_best)) 
print('验证集MSE：  ', abs(validation_score)) 

In [ ]:
print('*************************输出模型摘要信息*******************************')
print(lstm.summary()) 
 
plot_model(lstm, to_file='model.png', show_shapes=True) 

In [ ]:
def show_history(history):
        loss = history.history['loss']  
        val_loss = history.history['val_loss'] 
        epochs = range(1, len(loss) + 1)  
        plt.figure(figsize=(12, 4))  #
        plt.plot(epochs, loss, 'r', label='Training loss')  
        plt.plot(epochs, val_loss, 'b', label='Test loss')  
        plt.title('Training and Test loss')  
        plt.xlabel('Epochs') 
        plt.ylabel('Loss') 
        plt.legend()
        plt.show()  
 
 
show_history(history) 
 
y_pred = lstm.predict(X_test, batch_size=10)  
 
print('----------------模型评估-----------------')

print('**************************输出测试集的模型评估指标结果*******************************')
 
print('LSTM回归模型-最优参数-R^2:', round(r2_score(y_test, y_pred), 4))
print('LSTM回归模型-最优参数-均方误差:', round(mean_squared_error(y_test, y_pred), 4))
print('LSTM回归模型-最优参数-解释方差分:', round(explained_variance_score(y_test, y_pred), 4))
print('LSTM回归模型-最优参数-绝对误差:', round(mean_absolute_error(y_test, y_pred), 4))
 

plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False 
plt.plot(range(len(y_test)), y_test, color="blue", linewidth=1.5, linestyle="-") 
plt.plot(range(len(y_pred)), y_pred, color="red", linewidth=1.5, linestyle="-.")  
plt.legend(['真实值', '预测值']) 
plt.title("贝叶斯优化器优化LSTM回归模型真实值与预测值比对图") 
plt.show()  

### Final parameter of the classification model

In [ ]:
# RF
RF = RandomForestClassifier(n_estimators = 208, min_samples_split = 2, criterion = "gini", min_samples_leaf = 1, 
                            min_weight_fraction_leaf = 0, max_features = "sqrt", min_impurity_decrease = 0)
# LR
LR =  LogisticRegression(penalty = "l2", dual = False, tol = 0.0001, C = 1, verbose = 0)
# SVM
SVM = SVC(kernel='linear', probability= True, max_iter=787, degree=3, random_state=123, C = 1, tol = 0.001, cache_size = 200)
# XGBoost
XGBoost = xgb.XGBClassifier(n_estimators=503, gamma=13, learning_rate=0.01, n_jobs=8)
# GNB
GNB = GaussianNB(var_smoothing=3.9120442541811716e-08)
# DT
DT = DecisionTreeClassifier(max_depth=136, criterion = "gini", splitter = "best", min_samples_split = 2, 
                            min_samples_leaf = 1,min_weight_fraction_leaf = 0, min_impurity_decrease = 0)
# LSTM
lstm = Sequential() 
lstm.add(LSTM(units=32, return_sequences=True, input_shape=(X_trainf.shape[1], 1))) 
lstm.add(LSTM(units=int(53))) 
lstm.add(Dense(10, activation='relu')) 
lstm.add(Dense(1))  
lstm.compile(loss='mean_squared_error',
                 optimizer='sgd', metrics='mse') 

## Survival Model Hyperparametric Search

### Load data and variable selection

In [ ]:
X_train=X_train[[
'imv',
'age',
'aniongap',
'resp_rate',
'tracheostomy',
'apsiii',
'race_black',
'case_micu/sicu'
]]

X_test=X_test[[
'imv',
'age',
'aniongap',
'resp_rate',
'tracheostomy',
'apsiii',
'race_black',
'case_micu/sicu'
]]

In [ ]:
## Variable conversion
from sksurv.util import Surv
# training data
y_train_ = Surv.from_dataframe(
    event='status', 
    time='survival_time', 
    data=y_train)
y_train_
# testing data
y_test_ = Surv.from_dataframe(
    event='status', 
    time='survival_time', 
    data=y_test)

### BayesSearchCV for Cox EST ST RSF GBST

In [ ]:
### BayesSearchCV code for Cox EST ST RSF GBST
# EST example
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
search_spaces = {
  'n_estimators': Integer(..., ...),
  'max_depth': Integer(..., ...) 
}

n_iter_search = 20 
bayes_search = BayesSearchCV( 
    ExtraSurvivalTrees(), 
    search_spaces, 
    n_iter=n_iter_search, 
    cv=10, 
    verbose=3 
) 
bayes_search.fit(X_train, y_train_)

print("val. score: %s" % bayes_search.best_score_)
print("test score: %s" % bayes_search.score(X_test, y_test_))
print("best params: %s" % str(bayes_search.best_params_))

### BayesianOptimization for DeepSurv

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from pysurvival.models.simulations import SimulationModel
from pysurvival.models.semi_parametric import NonLinearCoxPHModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score

# Creating the X, Y and E input
Y_train = y_train['survival_time'].values
Y_test = y_test['survival_time'].values
E_train = (y_train['status'].values).astype(int)
E_test = (y_test['status'].values).astype(int)

In [ ]:
from bayes_opt import BayesianOptimization

# Given the extracted code, I will rewrite the Bayesian Optimization functions.

# Objective function for Bayesian optimization
def bayesopt_objective_deepsurv(unit):
    structure = [
        {'activation': 'relu', 'num_units': int(unit)}, 
        {'activation': 'tanh', 'num_units': 32}
    ]
    # You should train your model here and return the score
    deepsurv = NonLinearCoxPHModel(structure=structure)
    deepsurv.fit(X_train, Y_train, E_train, lr=1e-3, init_method='xav_uniform', num_epochs=200,dropout=0.8,batch_normalization=True,bn_and_dropout=True)

    score = concordance_index(deepsurv, X_test, Y_test, E_test)
    return score  
    return -unit  # Mock score (negative because BayesianOptimization tries to maximize the objective)

# Bayesian optimizer
def param_bayes_opt_deepsurv(init_points, n_iter):
    opt = BayesianOptimization(bayesopt_objective_deepsurv, {'unit': (..., ...)}, random_state=100, allow_duplicate_points=True)
    opt.maximize(init_points=init_points, n_iter=n_iter)
    params_best = opt.max['params']
    score_best = opt.max['target']
    return params_best, score_best



# Main function
def main_bayesian_optimization():
    param_grid_simple = {'unit': (..., ...)}
    params_best, score_best = param_bayes_opt_deepsurv(10, 10)
    print('Best parameter combination:', 'unit value:', int(params_best['unit']))

main_bayesian_optimization()

### Final parameter of the survival model

In [ ]:
# Cox
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
Cox = CoxPHSurvivalAnalysis(alpha = 0, ties = "breslow", n_iter = 100, tol = 1e-9, verbose = 0)
# EST
from sksurv.ensemble import ExtraSurvivalTrees
EST = ExtraSurvivalTrees(n_estimators=208, max_depth=100, min_samples_split = 6, min_samples_leaf = 3,
                         min_weight_fraction_leaf = 0, max_features = "sqrt")
# ST
from sksurv.tree import SurvivalTree
ST = SurvivalTree(max_depth=11, splitter = "best", max_depth = None, min_samples_split = 6, 
                  min_samples_leaf = 3, min_weight_fraction_leaf = 0)
# RSF
from sksurv.ensemble import RandomSurvivalForest 
RSF = RandomSurvivalForest(n_estimators=209,  max_depth = None, min_samples_split = 6, min_samples_leaf = 3, 
                           min_weight_fraction_leaf = 0, max_features = "sqrt")
# GBST
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
GBST =  GradientBoostingSurvivalAnalysis(n_estimators=204, max_depth=4, loss = "coxph", learning_rate = 0.1, criterion = 'friedman_mse',
                                         min_samples_split = 2, min_samples_leaf = 1, min_weight_fraction_leaf = 0)
# DeepSurv
structure = [ {'activation': 'relu', 'num_units': 116}, 
             {'activation': 'tanh', 'num_units': 32}, ] 
DeepSurv = NonLinearCoxPHModel(structure=structure)
DeepSurv.fit(
    X_train, Y_train, E_train, 
    lr=1e-3, 
    init_method='xav_uniform', 
    num_epochs=200,
    dropout=0.8,
    batch_normalization=True,
    bn_and_dropout=True
)